# E91 Implementation

Rory Schadler, Blake Danziger  
Physics 75  
Lab 2  

## Protocol Steps
1. For each bit of the message, Alice and Bob share an EPR pair.
2. Alice picks a random measurement basis from a = {0, pi/4, pi/2} and Bob picks a random measurement basis from b= {pi/4, pi/2, 3pi/4}. Alice and Bob each measure their piece of the EPR pair in their randomly chosen measurement basis
3. Alice and Bob anounce the measurement bases they used for each shared pair.
4. Alice and Bob announce the results of their measurements for the pairs where their measurement bases did not coincide.

## Implementation Steps
1. Create $n$ bell pairs, one for each message bit.
2. choose measurement bases for Alice, Bob
    * rotate $|0\rangle$, $|1\rangle$ by $a \in \{0,\frac{\pi}{4},\frac{\pi}{2}\}$ or $b \in \{\frac{\pi}{4},\frac{\pi}{2},\frac{3\pi}{4}\}$*
3. Alice and Bob each measure their bits
4. Find pairs where Alice and Bob used the same measurement basis
5. Compare measurements of bits where Alice's and Bob's measurements did not match

Source: [https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661)

In [63]:
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_histogram
from numpy.random import randint#, choice
import numpy as np
print("Imports Successful")

Imports Successful


In [64]:
msg_len = 10

In [65]:
def alice_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        alice_q = 2*i
        base = randint(3)
        chosen_bases.append(base+1)
        rot = np.pi * base / 4
        qc.u1(-rot,alice_q)
        qc.measure(alice_q,alice_q)
    return chosen_bases

def bob_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        bob_q = 2*i + 1
        base = randint(1,4)
        chosen_bases.append(base)
        rot = np.pi * base / 4
        qc.u1(-rot,bob_q)
        qc.measure(bob_q,bob_q)
    return chosen_bases

In [66]:
# Prepare Bell States
qc = QuantumCircuit(2*msg_len, 2*msg_len)
for i in range(msg_len):
    q1 = 2*i
    q2 = 2*i + 1
    qc.x(q2)
    qc.h(q1)
    qc.cx(q1,q2)
qc.barrier()
qc.draw()

┌───┐      ░ 
 q_0: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
 q_1: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
 q_2: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
 q_3: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
 q_4: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
 q_5: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
 q_6: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
 q_7: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
 q_8: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
 q_9: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
q_10: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
q_11: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
q_12: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
q_13: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
q_14: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
q_15: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
q_16: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
q_17: ┤ X ├┤ X ├─░─
      ├───┤└───┘ ░ 
q_18: ┤ H ├──■───░─
      ├───┤┌─┴─┐ ░ 
q_19: ┤ X ├┤ X ├─░─
      └───┘└───┘ ░ 
 c_0: ═════════════
                   
 c_1: ═════════════
                   
 c_2: ═════════════
                   
 c_3: ═════════════
                   
 c_4: ═════════════
                   
 c_5: ═════════════
                   
 c_6: ═════════════
                   
 c_7: ═════════════
                   
 c_8: ═════════════
                   
 c_9: ═════════════
                   
c_10: ═════════════
                   
c_11: ═════════════
                   
c_12: ═════════════
                   
c_13: ═════════════
                   
c_14: ═════════════
                   
c_15: ═════════════
                   
c_16: ═════════════
                   
c_17: ═════════════
                   
c_18: ═════════════
                   
c_19: ═════════════

In [67]:
# add measurements with random bases
a_bases = alice_measure(qc)
b_bases = bob_measure(qc)
qc.draw()

┌───┐      ░ ┌───────────┐ ┌─┐                                          »
 q_0: ┤ H ├──■───░─┤ U1(-pi/2) ├─┤M├──────────────────────────────────────────»
      ├───┤┌─┴─┐ ░ ├───────────┤ └╥┘                           ┌─┐            »
 q_1: ┤ X ├┤ X ├─░─┤ U1(-pi/2) ├──╫────────────────────────────┤M├────────────»
      ├───┤└───┘ ░ ├───────────┤  ║ ┌─┐                        └╥┘            »
 q_2: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫─┤M├─────────────────────────╫─────────────»
      ├───┤┌─┴─┐ ░ ├───────────┤  ║ └╥┘                         ║ ┌─┐         »
 q_3: ┤ X ├┤ X ├─░─┤ U1(-pi/2) ├──╫──╫──────────────────────────╫─┤M├─────────»
      ├───┤└───┘ ░ └─┬───────┬─┘  ║  ║ ┌─┐                      ║ └╥┘         »
 q_4: ┤ H ├──■───░───┤ U1(0) ├────╫──╫─┤M├──────────────────────╫──╫──────────»
      ├───┤┌─┴─┐ ░ ┌─┴───────┴─┐  ║  ║ └╥┘                      ║  ║ ┌─┐      »
 q_5: ┤ X ├┤ X ├─░─┤ U1(-pi/4) ├──╫──╫──╫───────────────────────╫──╫─┤M├──────»
      ├───┤└───┘ ░ ├───────────┤  ║  ║  ║ ┌─┐                   ║  ║ └╥┘      »
 q_6: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫──╫──╫─┤M├───────────────────╫──╫──╫───────»
      ├───┤┌─┴─┐ ░ ├───────────┤  ║  ║  ║ └╥┘                   ║  ║  ║ ┌─┐   »
 q_7: ┤ X ├┤ X ├─░─┤ U1(-pi/4) ├──╫──╫──╫──╫────────────────────╫──╫──╫─┤M├───»
      ├───┤└───┘ ░ ├───────────┤  ║  ║  ║  ║ ┌─┐                ║  ║  ║ └╥┘   »
 q_8: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫──╫──╫──╫─┤M├────────────────╫──╫──╫──╫────»
      ├───┤┌─┴─┐ ░ ├───────────┴┐ ║  ║  ║  ║ └╥┘                ║  ║  ║  ║ ┌─┐»
 q_9: ┤ X ├┤ X ├─░─┤ U1(-3pi/4) ├─╫──╫──╫──╫──╫─────────────────╫──╫──╫──╫─┤M├»
      ├───┤└───┘ ░ └─┬───────┬──┘ ║  ║  ║  ║  ║ ┌─┐             ║  ║  ║  ║ └╥┘»
q_10: ┤ H ├──■───░───┤ U1(0) ├────╫──╫──╫──╫──╫─┤M├─────────────╫──╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ┌─┴───────┴──┐ ║  ║  ║  ║  ║ └╥┘             ║  ║  ║  ║  ║ »
q_11: ┤ X ├┤ X ├─░─┤ U1(-3pi/4) ├─╫──╫──╫──╫──╫──╫──────────────╫──╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────┬┘ ║  ║  ║  ║  ║  ║ ┌─┐          ║  ║  ║  ║  ║ »
q_12: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫──╫──╫──╫──╫──╫─┤M├──────────╫──╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ├───────────┤  ║  ║  ║  ║  ║  ║ └╥┘          ║  ║  ║  ║  ║ »
q_13: ┤ X ├┤ X ├─░─┤ U1(-pi/2) ├──╫──╫──╫──╫──╫──╫──╫───────────╫──╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────┤  ║  ║  ║  ║  ║  ║  ║ ┌─┐       ║  ║  ║  ║  ║ »
q_14: ┤ H ├──■───░─┤ U1(-pi/2) ├──╫──╫──╫──╫──╫──╫──╫─┤M├───────╫──╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ├───────────┴┐ ║  ║  ║  ║  ║  ║  ║ └╥┘       ║  ║  ║  ║  ║ »
q_15: ┤ X ├┤ X ├─░─┤ U1(-3pi/4) ├─╫──╫──╫──╫──╫──╫──╫──╫────────╫──╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────┬┘ ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐    ║  ║  ║  ║  ║ »
q_16: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────╫──╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ├───────────┤  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘    ║  ║  ║  ║  ║ »
q_17: ┤ X ├┤ X ├─░─┤ U1(-pi/4) ├──╫──╫──╫──╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────┤  ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐ ║  ║  ║  ║  ║ »
q_18: ┤ H ├──■───░─┤ U1(-pi/4) ├──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─╫──╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ├───────────┤  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘ ║  ║  ║  ║  ║ »
q_19: ┤ X ├┤ X ├─░─┤ U1(-pi/2) ├──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      └───┘└───┘ ░ └───────────┘  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
 c_0: ════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬═»
                                     ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
 c_1: ═══════════════════════════════╬══╬══╬══╬══╬══╬══╬══╬══╬══╩══╬══╬══╬══╬═»
                                     ║  ║  ║  ║  ║  ║  ║  ║  ║     ║  ║  ║  ║ »
 c_2: ═══════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬══╬═════╬══╬══╬══╬═»
                                        ║  ║  ║  ║  ║  ║  ║  ║     ║  ║  ║  ║ »
 c_3: ══════════════════════════════════╬══╬══╬══╬══╬══╬══╬══╬═════╩══╬══╬══╬═»
                                        ║  ║  ║  ║  ║  ║  ║  ║        ║  ║  ║ »
 c_4: ══════════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬════════╬══╬══╬═»
      

In [68]:
qasm_sim = Aer.get_backend('qasm_simulator')
qobj = assemble(qc, shots=1, memory=True)
result = qasm_sim.run(qobj).result()
# get_counts returns a dict, with the classical register as the *key* 
# which feels silly. this ugly idiom retrieves that key. surely a 
# better way to do this
measurements = [k for k in result.get_counts()][0]

alice_m = ""
bob_m = ""
for i in range(2*msg_len):
    if not i%2:
        alice_m += measurements[i]
    else:
        bob_m += measurements[i]

print(alice_m)
print(bob_m)
print(measurements)

0010100001
1101011110
01011001100101010110


In [69]:
for idx, (a_b, b_b) in enumerate(zip(a_bases, b_bases)):
    if((a_b == 2 and b_b == 1) or (a_b == 3 and b_b == 2)):
        a_m = alice_m[idx]
        b_m = bob_m[idx]
        print("match! Alice measured {} and Bob measured {} on qubit {}".format(a_m, b_m, idx))

match! Alice measured 0 and Bob measured 1 on qubit 0
match! Alice measured 0 and Bob measured 1 on qubit 3
match! Alice measured 0 and Bob measured 1 on qubit 8
